In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rockinbrock/spy-1-minute-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/spy-1-minute-data


In [2]:
# ===================================================================
# BLOCK 1: Package Installation & Imports
# ===================================================================

import subprocess
import sys

print("📦 Installing required packages...")
packages = ['xgboost', 'lightgbm', 'scikit-learn', 'optuna']
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import copy
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports complete\n")

📦 Installing required packages...
✅ Imports complete



In [3]:
print("📂 STEP 1: DATA LOADING")
print("="*70)

data_dir = Path("/kaggle/input/spy-1-minute-data")
csv_file = sorted(data_dir.glob("*.csv"))[0]
print(f"Loading: {csv_file.name}")

df = pd.read_csv(csv_file)
print(f"Raw shape: {df.shape}")

df.columns = [c.lower() for c in df.columns]

if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
elif 'date' in df.columns:
    df['datetime'] = pd.to_datetime(df['date'])

df = df.set_index('datetime').sort_index()

keep = [c for c in ['open', 'high', 'low', 'close', 'volume'] if c in df.columns]
X_raw = df[keep].copy()

print(f"Clean shape: {X_raw.shape}")
print(f"Date range: {X_raw.index.min()} to {X_raw.index.max()}")
print("✅ Step 1 Complete\n")


📂 STEP 1: DATA LOADING
Loading: spy_1min_2008_2021_cleaned.csv
Raw shape: (2070834, 8)
Clean shape: (2070834, 5)
Date range: 2008-01-22 07:30:00 to 2021-05-06 13:59:00
✅ Step 1 Complete



In [4]:
 
print("="*70)
print("🏷  STEP 2: LABEL GENERATION")
print("="*70)

H = 15
print(f"Horizon: {H} minutes")

ret = X_raw['close'].shift(-H).div(X_raw['close']).sub(1.0)
y = (ret > 0.0).astype(int)

valid = y.notna()
X_raw = X_raw.loc[valid]
y = y.loc[valid]

print(f"Valid samples: {len(y):,}")
print(f"Label distribution:")
print(y.value_counts(normalize=True))

print("\n📅 Creating events structure...")
events = pd.DataFrame(index=X_raw.index)
events['t1'] = X_raw.index + pd.Timedelta(minutes=H)
max_time = X_raw.index[-1]
events['t1'] = events['t1'].where(events['t1'] <= max_time, max_time)

print(f"Events created: {len(events):,}")
print("✅ Step 2 Complete\n")


🏷  STEP 2: LABEL GENERATION
Horizon: 15 minutes
Valid samples: 2,070,834
Label distribution:
close
0    0.503216
1    0.496784
Name: proportion, dtype: float64

📅 Creating events structure...
Events created: 2,070,834
✅ Step 2 Complete



In [5]:
print("=" * 70)
print("🔄 STEP 3: PURGED K-FOLD CV")
print("=" * 70)

class PurgedKFoldEmbargo:
    def __init__(self, n_splits=5, embargo_pct=0.01):  # ✅ double underscores
        assert n_splits >= 2
        assert 0.0 <= embargo_pct < 1.0
        self.n_splits = n_splits
        self.embargo_pct = embargo_pct
        print(f"Initialized: K={n_splits}, Embargo={embargo_pct*100:.1f}%")

    def split(self, X, y=None, groups=None, events=None):
        idx = X.index
        n = len(idx)

        # Compute fold sizes
        fold_sizes = np.full(self.n_splits, n // self.n_splits, dtype=int)
        fold_sizes[: n % self.n_splits] += 1

        # Compute fold edges
        edges = []
        cur = 0
        for fs in fold_sizes:
            edges.append((cur, cur + fs))
            cur += fs

        embargo = int(np.ceil(self.embargo_pct * n))

        # Map events to index positions
        ev = None
        if events is not None:
            ev = events.copy()
            ev = ev.loc[ev.index.intersection(idx)]
            max_time = idx[-1]
            ev['t1'] = ev['t1'].where(ev['t1'] <= max_time, max_time)
            pos = pd.Series(np.arange(n), index=idx)
            ev['start_pos'] = pos.reindex(ev.index).fillna(-1).astype(int)
            ev['end_pos'] = np.searchsorted(idx.values, ev['t1'].values, side="right") - 1
            ev['end_pos'] = ev['end_pos'].clip(0, n - 1)
            ev = ev[ev['start_pos'] >= 0]

        # Yield train/test indices
        for fold_idx, (a, b) in enumerate(edges):
            test_idx = np.arange(a, b, dtype=int)
            mask = np.ones(n, dtype=bool)
            L = max(0, a - embargo)
            R = min(n, b + embargo)
            mask[L:R] = False

            # Apply purge logic
            if ev is not None:
                ts, te = a, b - 1
                overlap = (ev['start_pos'] <= te) & (ev['end_pos'] >= ts)
                if overlap.any():
                    pL = int(ev.loc[overlap, 'start_pos'].min())
                    pR = int(ev.loc[overlap, 'end_pos'].max())
                    mask[pL:pR + 1] = False

            mask[test_idx] = False
            train_idx = np.flatnonzero(mask)

            yield train_idx, test_idx


# Initialize Purged K-Fold with Embargo
cv = PurgedKFoldEmbargo(n_splits=5, embargo_pct=0.01)
print("✅ Step 3 Complete\n")


🔄 STEP 3: PURGED K-FOLD CV
Initialized: K=5, Embargo=1.0%
✅ Step 3 Complete



In [6]:
 
# STEP 4: Feature Engineering
# ===================================================================
print("="*70)
print("🔧 STEP 4: FEATURE ENGINEERING")
print("="*70)

X = pd.DataFrame(index=X_raw.index)

# Returns
X['ret_1'] = X_raw['close'].pct_change(1)
X['ret_5'] = X_raw['close'].pct_change(5)
X['ret_15'] = X_raw['close'].pct_change(15)

# Price ranges
X['hl_range'] = (X_raw['high'] - X_raw['low']) / X_raw['close'].shift(1)
X['co_range'] = (X_raw['close'] - X_raw['open']) / X_raw['open']

# Volatility
X['vol_roll5'] = X['ret_1'].rolling(5).std()
X['vol_roll10'] = X['ret_1'].rolling(10).std()

# Volume
X['vol_z_30'] = ((X_raw['volume'] - X_raw['volume'].rolling(30).mean()) / 
                  X_raw['volume'].rolling(30).std())

# RSI
def compute_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    rsi = 100 - (100 / (1 + rs))
    return rsi

X['rsi_14'] = compute_rsi(X_raw['close'], period=14)

# Moving averages
X['sma_5'] = X_raw['close'].rolling(5).mean()
X['sma_20'] = X_raw['close'].rolling(20).mean()
X['price_sma5_ratio'] = X_raw['close'] / X['sma_5']

X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
y = y.loc[X.index]

print(f"Features created: {X.shape[1]}")
print("✅ Step 4 Complete\n")

🔧 STEP 4: FEATURE ENGINEERING
Features created: 12
✅ Step 4 Complete



In [7]:
def clone(model): 
    return copy.deepcopy(model)

# Pick ONE model to start (others commented)
rf = RandomForestClassifier(
    n_estimators=300,         # lower to speed up; raise later
    max_depth=None,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced_subsample',
    n_jobs=-1,
    random_state=42
)

xgb = XGBClassifier(
    n_estimators=400,         # lower for speed; use early stopping later
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    n_jobs=-1,
    tree_method='hist',
    eval_metric='logloss',
    random_state=42
)

lgb = LGBMClassifier(
    n_estimators=600,         # lower for speed; use early stopping later
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    n_jobs=-1,
    objective='binary',
    random_state=42,
)

MODEL_NAME = "RandomForest"     # ← change to "XGBoost" or "LightGBM" later
MODEL = {"RandomForest": rf, "XGBoost": xgb, "LightGBM": lgb}[MODEL_NAME]


In [8]:
from math import isfinite

def fit_predict_fold(model, Xtr, ytr, Xte, yte):
    m = clone(model)
    # early stopping for boosters
    if isinstance(m, XGBClassifier):
        m.set_params(n_estimators=2000)  # allow more, but ES will cut
        m.fit(Xtr, ytr, eval_set=[(Xte, yte)], verbose=False, early_stopping_rounds=100)
    elif isinstance(m, LGBMClassifier):
        m.set_params(n_estimators=2000)  # allow more, but ES will cut
        m.fit(Xtr, ytr, eval_set=[(Xte, yte)], verbose=False, early_stopping_rounds=100)
    else:
        m.fit(Xtr, ytr)
    p = m.predict_proba(Xte)[:, 1]
    pred = (p >= 0.5).astype(int)
    auc = roc_auc_score(yte, p)
    acc = accuracy_score(yte, pred)
    return auc, acc

print("="*70)
print(f"Running ONE fold for {MODEL_NAME} ...")

folds = list(cv.split(X, events=events))
(tr, te) = folds[0]  # first fold only
auc, acc = fit_predict_fold(MODEL, X.iloc[tr], y.iloc[tr], X.iloc[te], y.iloc[te])
print(f"Fold 1 → AUC={auc:.4f} | ACC={acc:.4f}")


Running ONE fold for RandomForest ...
Fold 1 → AUC=0.4970 | ACC=0.5011


In [9]:
START_FOLD = 0   # 0-based
END_FOLD   = 2   # not inclusive; so folds 0 and 1

aucs, accs = [], []
for fi, (tr, te) in enumerate(folds[START_FOLD:END_FOLD], start=START_FOLD+1):
    auc, acc = fit_predict_fold(MODEL, X.iloc[tr], y.iloc[tr], X.iloc[te], y.iloc[te])
    aucs.append(auc); accs.append(acc)
    print(f"Fold {fi} → AUC={auc:.4f} | ACC={acc:.4f}")

print("-"*70)
print(f"{MODEL_NAME} PARTIAL → AUC={np.mean(aucs):.4f}±{np.std(aucs):.4f} | ACC={np.mean(accs):.4f}±{np.std(accs):.4f}")


Fold 1 → AUC=0.4970 | ACC=0.5011
Fold 2 → AUC=0.5014 | ACC=0.5026
----------------------------------------------------------------------
RandomForest PARTIAL → AUC=0.4992±0.0022 | ACC=0.5018±0.0007


In [10]:
dup_count = X.index.duplicated().sum()
print(f"Duplicate timestamps in X: {dup_count}")

dup_count_y = y.index.duplicated().sum()
print(f"Duplicate timestamps in y: {dup_count_y}")


Duplicate timestamps in X: 638054
Duplicate timestamps in y: 2656940


In [11]:
# See sample duplicate timestamps
dup_times = X.index[X.index.duplicated()].unique()[:10]
print("Example duplicate times in X:\n", dup_times)
print("\nCount of rows per duplicate timestamp:")
print(X.loc[dup_times].groupby(level=0).size().head())


Example duplicate times in X:
 DatetimeIndex(['2008-01-25 07:30:00', '2008-01-25 07:31:00',
               '2008-01-25 07:32:00', '2008-01-25 07:33:00',
               '2008-01-25 07:34:00', '2008-01-25 07:35:00',
               '2008-01-25 07:36:00', '2008-01-25 07:37:00',
               '2008-01-25 07:38:00', '2008-01-25 07:39:00'],
              dtype='datetime64[ns]', name='datetime', freq=None)

Count of rows per duplicate timestamp:
datetime
2008-01-25 07:30:00    3
2008-01-25 07:31:00    3
2008-01-25 07:32:00    3
2008-01-25 07:33:00    3
2008-01-25 07:34:00    3
dtype: int64


In [12]:
# Aggregate triplicate minute bars into single bar per minute
X_agg = X_raw.groupby(X_raw.index).agg({
    'open': 'first',     # first open of the minute
    'high': 'max',       # highest high
    'low': 'min',        # lowest low
    'close': 'last',     # last close
    'volume': 'sum'      # total traded volume
})


In [13]:
print(f"Before aggregation: {X_raw.shape}")
print(f"After aggregation:  {X_agg.shape}")
print(f"Duplicate timestamps after agg: {X_agg.index.duplicated().sum()}")


Before aggregation: (2070834, 5)
After aggregation:  (1432780, 5)
Duplicate timestamps after agg: 0


In [14]:
H = 15  # 15-minute horizon
ret = X_agg['close'].shift(-H).div(X_agg['close']).sub(1.0)
y = (ret > 0.0).astype(int).dropna()


In [15]:
X_agg = X_agg.loc[y.index]


In [16]:
events = pd.DataFrame(index=X_agg.index)
events['t1'] = X_agg.index + pd.Timedelta(minutes=H)
max_time = X_agg.index[-1]
events['t1'] = events['t1'].where(events['t1'] <= max_time, max_time)


In [17]:
X = pd.DataFrame(index=X_agg.index)

# Returns
X['ret_1'] = X_agg['close'].pct_change(1)
X['ret_5'] = X_agg['close'].pct_change(5)
X['ret_15'] = X_agg['close'].pct_change(15)

# Price ranges
X['hl_range'] = (X_agg['high'] - X_agg['low']) / X_agg['close'].shift(1)
X['co_range'] = (X_agg['close'] - X_agg['open']) / X_agg['open']

# Volatility
X['vol_roll5'] = X['ret_1'].rolling(5).std()
X['vol_roll10'] = X['ret_1'].rolling(10).std()

# Volume
X['vol_z_30'] = (X_agg['volume'] - X_agg['volume'].rolling(30).mean()) / X_agg['volume'].rolling(30).std()

# RSI
def compute_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    rsi = 100 - (100 / (1 + rs))
    return rsi

X['rsi_14'] = compute_rsi(X_agg['close'], period=14)

# Moving averages
X['sma_5'] = X_agg['close'].rolling(5).mean()
X['sma_20'] = X_agg['close'].rolling(20).mean()
X['price_sma5_ratio'] = X_agg['close'] / X['sma_5']

# Cleanup
X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
y = y.loc[X.index]


In [18]:
print(f"Final Shapes — X: {X.shape}, y: {y.shape}")
print(f"Duplicate timestamps left: {X.index.duplicated().sum()}")
assert len(X) == len(y)


Final Shapes — X: (1432780, 12), y: (1432780,)
Duplicate timestamps left: 0


In [19]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    tree_method='hist'
)

xgb_model.fit(X_sample, y_sample, verbose=False)
xgb_importance = pd.Series(xgb_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(xgb_importance.head(10))


NameError: name 'X_sample' is not defined

In [ ]:
print("="*70)
print("📊 STEP 6B: FEATURE IMPORTANCE — Permutation (MDA)")
print("="*70)

from sklearn.inspection import permutation_importance

# Use the same trained XGBoost model
mda_result = permutation_importance(
    xgb_model,
    X_sample,
    y_sample,
    scoring='roc_auc',
    n_repeats=3,
    random_state=42,
    n_jobs=-1
)

mda = pd.Series(mda_result.importances_mean, index=X.columns).sort_values(ascending=False)
print(mda.head(10))


In [ ]:
print("="*70)
print("📊 STEP 6C: FEATURE IMPORTANCE — Orthogonal (OFI)")
print("="*70)

from sklearn.linear_model import LinearRegression

def orthogonalize_features(X):
    X_ortho = pd.DataFrame(index=X.index)
    for col in X.columns:
        others = [c for c in X.columns if c != col]
        if len(others) == 0:
            X_ortho[col] = X[col]
            continue
        model = LinearRegression().fit(X[others], X[col])
        residual = X[col] - model.predict(X[others])
        X_ortho[col] = residual
    return X_ortho

X_ortho_sample = orthogonalize_features(X_sample)

xgb_model_ofi = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    tree_method='hist'
)

xgb_model_ofi.fit(X_ortho_sample, y_sample, verbose=False)
ofi = pd.Series(xgb_model_ofi.feature_importances_, index=X.columns).sort_values(ascending=False)
print(ofi.head(10))


In [ ]:
import matplotlib.pyplot as plt

# Combine importances
imp_df = pd.concat([
    xgb_importance.rename("MDI"),
    mda.rename("MDA"),
    ofi.rename("OFI")
], axis=1).fillna(0)

top_features = imp_df.mean(axis=1).sort_values(ascending=False).head(12).index

imp_df.loc[top_features].plot(kind="bar", figsize=(12,6))
plt.title("Feature Importance Comparison — XGBoost (MDI vs MDA vs OFI)")
plt.ylabel("Importance Score")
plt.tight_layout()
plt.show()


In [ ]:
plt.savefig("feature_importance_comparison.png", dpi=300)


In [ ]:
import seaborn as sns

plt.figure(figsize=(10,6))
sns.heatmap(X_sample.corr(), cmap="coolwarm", center=0)
plt.title("Feature Correlation Heatmap — Before Orthogonalization")
plt.show()
